In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import numpy as np
import nltk

In [5]:
print(np.__version__)
print(nltk.__version__)

1.20.1
3.6.2


In [7]:
import re
import os
import tarfile

from six.moves import urllib

In [6]:
DOWNLOADED_FILENAME = 'ImdbReviews.tar.gz'

def download_file(url_path):
    if not os.path.exists(DOWNLOADED_FILENAME):
        filename, _ = urllib.request.urlretrieve(url_path, DOWNLOADED_FILENAME)
        
    print('Found and verified file from this path: ', url_path)
    print('Downloaded file: ', DOWNLOADED_FILENAME)

In [14]:
TOKEN_REGEX = re.compile("[^A-Za-z0-9]+")

def get_reviews(dirname, positive=True):
    label = 1 if positive else 0
    
    reviews = []
    
    for filename in os.listdir(dirname):
        if filename.endswith(".txt"):
            
            with open(dirname + filename, 'r+', encoding='utf-8') as f:
                
                review = f.read()
                review = review.lower().replace("<br />", " ")
                
                # Return a tuple of the review text and a label for whether it
                # is a positive or negative review
                reviews.append((review, label))
    return reviews

def extract_reviews():
    
    # If the file has not already been extracted
    if not os.path.exists('aclImdb'):
        with tarfile.open(DOWNLOADED_FILENAME) as tar:
            tar.extractall()
            tar.close()
            
    positive_reviews = get_reviews("aclImdb/train/pos/", positive=True)
    negative_reviews = get_reviews("aclImdb/train/neg/", positive=False)
    
    return positive_reviews, negative_reviews

In [9]:
URL_PATH = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

download_file(URL_PATH)

Found and verified file from this path:  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Downloaded file:  ImdbReviews.tar.gz


In [15]:
positive_reviews, negative_reviews = extract_reviews()

In [ ]:
len(positive_reviews)

In [ ]:
len(negative_reviews)

In [ ]:
positive_reviews[:2]

In [ ]:
TRAIN_DATA = 5000
TOTAL_DATA = 6000

train_reviews = positive_reviews[:TRAIN_DATA] + negative_reviews[:TRAIN_DATA]

test_positive_reviews = positive_reviews[TRAIN_DATA:TOTAL_DATA]
test_negative_reviews = negative_reviews[TRAIN_DATA:TOTAL_DATA]

In [ ]:
def get_vocabulary(train_reviews):
    words_set = set()
    
    for review in train_reviews:
        words_set.update(review[0].split())
        
    return list(words_set)

vocabulary = get_vocabulary(train_reviews)

In [ ]:
len(vocabulary)

In [ ]:
vocabulary[:5]

In [ ]:
def extract_features(review_text):
    
    # Split the review into words, and create a set of the words
    review_words = set(review_text.split())
    
    features = {}
    for word in vocabulary:
        features[word] = (word in review_words)
        
    return features

In [ ]:
train_features = nltk.classify.apply_features(extract_features, train_reviews)

In [ ]:
trained_classifier = nltk.NaiveBayesClassifier.train(train_features)

In [ ]:
def sentiment_calculator(review_text):
    features = extract_features(review_text)
    return trained_classifier.classify(features)

In [ ]:
sentiment_calculator("What an amazing movie!")

In [ ]:
sentiment_calculator("What a terrible movie")

In [ ]:
def classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator):
    
    positive_results = [sentiment_calculator(review[0]) for review in test_positive_reviews]
    negative_results = [sentiment_calculator(review[0]) for review in test_negative_reviews]
    
    true_positives = sum(x > 0 for x in positive_results)
    true_negatives = sum(x == 0 for x in negative_results)
    
    percent_true_positive = float(true_positives) / len(positive_results)
    percent_true_negative = float(true_negatives) / len(negative_results)
    
    total_accurate = true_positives + true_negatives
    total = len(positive_results) + len(negative_results)
    
    print("Accuracy on posititve reviews = " +"%.2f" % (percent_true_positive * 100) + "%")
    print("Accuracy on negative reviews = " +"%.2f" % (percent_true_negative * 100) + "%")
    print("Overall accuracy = " +"%.2f" % (total_accurate * 100/ total) + "%")

In [ ]:
classify_test_reviews(test_positive_reviews, test_negative_reviews, sentiment_calculator)